<a href="https://colab.research.google.com/github/vlad98rus/-netology_pyda/blob/HomeWork-1/%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%86%D0%B8%D0%B8_%D0%BD%D0%B0_%D0%BE%D1%81%D0%BD%D0%BE%D0%B2%D0%B5_%D1%81%D0%BA%D1%80%D1%8B%D1%82%D1%8B%D1%85_%D1%84%D0%B0%D0%BA%D1%82%D0%BE%D1%80%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание «Рекомендации на основе скрытых факторов»

-  “Требуется построить модель рекомендаций на основе скрытых факторов (implicit) на основе dataset’а https://grouplens.org/datasets/hetrec-2011/ (Delicious Bookmarks)”

In [35]:
pip install implicit

In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import scipy.sparse as sparse
from implicit.als import AlternatingLeastSquares

In [0]:
raw_data = pd.read_table('user_taggedbookmarks-timestamps.dat', usecols=['userID','bookmarkID','tagID'])

In [9]:
print(raw_data.describe())

              userID     bookmarkID          tagID
count  107022.000000  107021.000000  107021.000000
mean     7476.520996   14574.969296    2962.185992
std      4170.516443    9251.971464    4380.586388
min         8.000000       1.000000       1.000000
25%      3646.000000    6501.000000     324.000000
50%      7506.000000   13532.000000     946.000000
75%     11295.000000   22356.000000    3307.000000
max     14323.000000   32167.000000   19127.000000


In [8]:
raw_data.head()

,userID,bookmarkID,tagID
0,8,1.0,1.0
1,8,2.0,1.0
2,8,7.0,1.0
3,8,7.0,6.0
4,8,7.0,7.0


In [10]:
# подсчитаем количество tag-ов (TagCount) вместо tagID
raw_data = raw_data.groupby(['userID', 'bookmarkID'])['tagID'].count().reset_index()
raw_data.columns = ['userID', 'bookmarkID', 'tagCount']
raw_data.head()

,userID,bookmarkID,tagCount
0,8,1.0,1
1,8,2.0,1
2,8,7.0,3
3,8,8.0,3
4,8,9.0,2


In [0]:
rows = raw_data.userID.astype(int)
cols = raw_data.bookmarkID.astype(int)
tags = list(raw_data.tagCount)

In [0]:
data_sparse = sparse.csr_matrix((tags, (cols, rows)), shape=(max(cols)+1, max(rows)+1))

In [29]:
model = AlternatingLeastSquares(factors=100,use_gpu=False)
model.fit(data_sparse)

In [30]:
list(raw_data['userID'].unique())

[8,
 32,
 57,
 147,
 233,
 242,
 259,
 302,
 319,
 336,
 372,
 427,
 485,
 521,
 538,
 620,
 788,
 819,
 913,
 934,
 970,
 1016,
 1033,
 1049,
 1094,
 1103,
 1113,
 1149,
 1263,
 1296,
 1320,
 1328,
 1352,
 1423,
 1455,
 1476,
 1505,
 1557,
 1701,
 1751,
 1774,
 1822,
 1856,
 1887,
 1933,
 1952,
 1987,
 2022,
 2032,
 2055,
 2128,
 2151,
 2232,
 2300,
 2315,
 2330,
 2367,
 2393,
 2437,
 2462,
 2487,
 2498,
 2549,
 2570,
 2586,
 2611,
 2636,
 2701,
 2716,
 2748,
 2831,
 2879,
 2911,
 2934,
 2952,
 2961,
 2989,
 3032,
 3083,
 3122,
 3203,
 3220,
 3233,
 3257,
 3279,
 3337,
 3396,
 3421,
 3454,
 3513,
 3539,
 3568,
 3646,
 3728,
 3743,
 3789,
 3817,
 3836,
 3914,
 3943,
 3957,
 4031,
 4089,
 4109,
 4135,
 4154,
 4188,
 4237,
 4324,
 4363,
 4488,
 4549,
 4681,
 4717,
 4742,
 4762,
 4795,
 4828,
 4911,
 4977,
 5024,
 5046,
 5075,
 5108,
 5186,
 5251,
 5266,
 5323,
 5424,
 5478,
 5584,
 5606,
 5648,
 5802,
 5817,
 5863,
 5937,
 5947,
 5959,
 6020,
 6047,
 6067,
 6092,
 6111,
 6120,
 6141,
 61

In [31]:
userid = 259

user_items = data_sparse.T.tocsr()
recommendations = model.recommend(userid, user_items)
print(recommendations)

[(15812, 0.37734878), (24991, 0.3537666), (24825, 0.35337257), (4948, 0.3369683), (24990, 0.32255557), (24989, 0.32254803), (24955, 0.32254285), (24951, 0.3225345), (24972, 0.32253075), (24933, 0.32252747)]


In [32]:
titles = pd.read_table('bookmarks.dat', encoding='koi8-r', usecols=['id','title'])
titles.head()

,id,title
0,1,IFLA - The official website of the Internation...
1,2,gcdp-e.pdf (application/pdf Object)
2,7,EdSelect
3,8,Cool Canada (Collections Canada)
4,9,Kidsreads.com


In [36]:
# Вывод: наша модель говорит нам о необходимости внести в закладки следующие страницы:
titles[titles['id'].isin([r[0] for r in recommendations])]

,id,title
3504,4948,"An introduction to eventmachine, and how to av..."
11082,15812,Ruby &amp; WebSockets: TCP for the Browser - i...
17028,24825,BackTrack Linux
17109,24933,Vuvuzela Filter using Fedora | Felix&#039; Blog
17126,24951,Labuat
17128,24955,How NodeJS saved my web application - publishi...
17138,24972,Teach Yourself Programming in Ten Years
17149,24989,A Platform Game Using HTML5 Canvas ╚ PurpleFlo...
17150,24990,Page Speed Home
17151,24991,Closure Tools - Google Code
